In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

print('Python version: ', sys.version_info)
print('TF version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

**1. Data Fetching and Loading**

In [ ]:
# Download the Flowers dataset: https://www.kaggle.com/datasets/imsparsh/flowers-dataset

# Create folder flower_photos
# Inside this folder there are 5 subfolders, one for each category

!curl -O https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz

!tar -xzvf flower_photos.tgz

!rm flower_photos.tgz



In [ ]:
os.chdir('flower_photos')

path = os.getcwd()



In [ ]:
# Check the total number of images

import pathlib

data_dir = pathlib.Path(path)

image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
# Creation of the dataset objects
# The images in the folders are not divided in train and validation datasets
# The following code divides samples into 80% training and 20% validation. No test set is created
# Check details from previous class and also here: https://www.tensorflow.org/tutorials/load_data/images


batch_size = 32
IMG_SIZE = (180, 180)

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=batch_size,
)

class_names = train_ds.class_names


train_ds = train_ds.cache().prefetch(1)
val_ds = val_ds.cache().prefetch(1)



In [ ]:
# Dataset detailed information

print('Nr. of classes: ', len(class_names))
print('Classes: ', class_names)

# Cardinality
print('Cardinalidade Treino: ', train_ds.cardinality().numpy())
print('Cardinalidade Validacão: ', val_ds.cardinality().numpy())



# Explain the cardinality values

In [ ]:
# Visualize a few examples

plt.figure(figsize=(12, 12))
for images, labels in train_ds.take(1):
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy()/255.)
        plt.title(class_names[labels[i]])
        plt.axis("off")

**2.	Creating the baseline version of the Neural Network using the Keras Functional API**

In [ ]:
# Creation of a baseline CNN

# It must comply with the following constraints:
#   1. Use Keras Functional API
#   2. Use only Conv2D, MaxPooling, Dense, Flatten and Rescaling layers
#   3. Maximum of 5 million parameters
#   4. Without Data Augmentation
#   5. Do not forget Input Rescaling

# https://www.tensorflow.org/api_docs/python/tf/keras/Model
# https://www.tensorflow.org/guide/keras/sequential_model
# https://keras.io/api/layers/

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# FUNCTIONAL API

inputs = keras.Input(shape=(180,180,3))

a = layers.Rescaling(scale = 1./255)(inputs)


  ## Complete the Model


outputs =  ## Complete ##

model = keras.Model(inputs=inputs, outputs=outputs)




In [ ]:
# Check the limit for the maximum number of parameters

model.summary()

In [ ]:
# Model compilation

#  1. Select the loss function suitable for this situation
#  2. Adopt ADAM optimizer, with default parameterization
#  3. Select accuracy metric to evaluate the model

L = ## Complete ##

model.compile(loss=L, optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train for 20 epochs

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20
)

In [ ]:
# Visualize results (both accuracy and loss)

import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

**Question:**

1. Analyze results. By looking at each one of the charts, what do you think is happening?

2. Suggest possible strategies to enhance results.


**3.	Fighting Overfitting: Data Augmentation**

In [ ]:

# Overview on Data Augmentation:
# https://www.tensorflow.org/tutorials/images/data_augmentation

# Model DA1 relies on Keras preprocessing layers to perform Data Augmentation
# https://keras.io/api/layers/preprocessing_layers/image_augmentation/

# The data augmentation module is created using the Sequential API

data_augmentation1 = keras.Sequential([
    layers.RandomFlip(mode='horizontal'),
    layers.RandomRotation(factor=0.5),
])




In [ ]:
# Visualize examples of augmented images

plt.figure(figsize=(20, 20))
for images, _ in train_ds.take(1):
  for i in range(16):
    augmented_images = data_augmentation1(images)
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(augmented_images[0].numpy()/255.)
    plt.axis("off")

In [ ]:
# Model DA1 with data augmentation

# Keep the previous model and just add the data augmentation layers

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# FUNCTIONAL API

inputs = keras.Input(shape=(180,180,3))

  ## Complete the model ##

model_DA1 = keras.Model(inputs=inputs, outputs=outputs)






In [ ]:
# Compile and train Model 2

L = ## Complete ##

model_DA1.compile(loss=L, optimizer="adam", metrics=["accuracy"])


history = model_DA1.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20
)

In [ ]:
# Visualize results (both accuracy and loss)

import pandas as pd
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

**Question:**

1. How do you analyze results? Are they similar to those obtained by the previous model?


In [ ]:
# Create a new data augmentation module - Model DA2

# It should comprise 3 or more pre-processing layers, where, at least, two of them,
# must be different from the ones already used

# Add the pre-processing module to the beggining of the NN
# Compile and train the modified CNN and analyze results

# Constraint: 5 million trainable parameters

# You can use additional strategies to fight overfitting


# CODE GOES HERE - MODEL DA2

